In [1]:
from natasha import NamesExtractor

In [2]:
sents = open('text.txt').read().splitlines()

In [3]:
# extractor = NamesExtractor('\n'.join(sents))

In [4]:
from yargy import Parser, rule
from yargy.tokenizer import MorphTokenizer
from yargy.pipelines import morph_pipeline
from yargy.interpretation import fact
from IPython.display import display
from yargy.predicates import normalized, caseless, gram

In [5]:
Person = fact(
    'Person',
    ['position', 'name']
)

Name = fact(
    'Name',
    ['first', 'last']
)


POSITION = morph_pipeline([
    'энтомолог',
    'президент',
    'жена'
])

NAME = rule(
    gram('Name').interpretation(
        Name.first.inflected()
    ),
    gram('Surn').interpretation(
        Name.last.inflected()
    )
).interpretation(
    Name
)

PERSON = rule(
    POSITION.interpretation(
        Person.position.inflected()
    ),
    NAME.interpretation(
        Person.name
    )
).interpretation(
    Person
)


parser = Parser(PERSON)

In [6]:
matches = []

for sent in sents:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [7]:
matches

[Person(
     position='жена',
     name=Name(
         first='милёва',
         last='марич'
     )
 ),
 Person(
     position='энтомолог',
     name=Name(
         first='кристофер',
         last='ауривиллиус'
     )
 ),
 Person(
     position='президент',
     name=Name(
         first='франклин',
         last='рузвельт'
     )
 )]

In [8]:
from yargy import not_
from yargy.predicates import eq

Item = fact(
    'Item',
    ['titles']
)

TITLE = rule(
    '«',
    not_(eq('»')).repeatable(),
    '»'
)
ITEM = rule(
    TITLE.interpretation(
        Item.titles
    )
).interpretation(
    Item
)

parser = Parser(ITEM)
matches = []

for sent in sents:
    for match in parser.findall(sent):
        matches.append(match.fact)

In [9]:
matches

[Item(
     titles='«квантовую телепортацию»'
 ),
 Item(
     titles='«Начала»'
 ),
 Item(
     titles='«Критику чистого разума»'
 ),
 Item(
     titles='«Анналы физики»'
 ),
 Item(
     titles='«Следствия теории капиллярности»'
 ),
 Item(
     titles='«Анналы физики»'
 ),
 Item(
     titles='«Год чудес»'
 ),
 Item(
     titles='«Год чудес»'
 ),
 Item(
     titles='«Анналы физики»'
 ),
 Item(
     titles='«К электродинамике движущихся тел»'
 ),
 Item(
     titles='«Об одной эвристической точке зрения, касающейся возникновения и превращения света»'
 ),
 Item(
     titles='«О движении взвешенных в покоящейся жидкости частиц, требуемом молекулярно-кинетической теорией теплоты»'
 ),
 Item(
     titles='«эфирного ветра»'
 ),
 Item(
     titles='«эфирный ветер»'
 ),
 Item(
     titles='«конвенцию»'
 ),
 Item(
     titles='«К электродинамике движущихся тел»'
 ),
 Item(
     titles='«специальная теория относительности»'
 ),
 Item(
     titles='«новую физику»'
 ),
 Item(
     titles='«парадокс 

In [10]:
from yargy import Parser, rule, and_, or_
from yargy.interpretation import fact, attribute
from yargy.predicates import dictionary, gte, lte

Date = fact(
    'Date',
    ['year', 'month', 'day']
)


MONTHS = {
    'январь',
    'февраль',
    'март',
    'апрель',
    'мая',
    'июнь',
    'июль',
    'август',
    'сентябрь',
    'октябрь',
    'ноябрь',
    'декабрь'
}


MONTH_NAME = dictionary(MONTHS)
DAY = and_(
    gte(1),
    lte(31)
)
YEAR = and_(
    gte(1800),
    lte(2100)
)
DATE = rule(
    DAY.interpretation(
        Date.day
    ),
    MONTH_NAME.interpretation(
        Date.month
    ),
    YEAR.interpretation(
        Date.year
    ).optional()
).interpretation(
    Date
)



parser = Parser(DATE)
matches = []
for sent in sents:
    for match in parser.findall(sent):
        matches.append(match.fact)
matches

[Date(
     year='1879',
     month='марта',
     day='14'
 ),
 Date(
     year='1955',
     month='апреля',
     day='18'
 ),
 Date(
     year='1934',
     month='марта',
     day='24'
 ),
 Date(
     year='1879',
     month='марта',
     day='14'
 ),
 Date(
     year='1902',
     month='февраля',
     day='1'
 ),
 Date(
     year='1905',
     month='апреля',
     day='30'
 ),
 Date(
     year='1906',
     month='января',
     day='15'
 ),
 Date(
     year='1922',
     month='ноября',
     day='10'
 ),
 Date(
     year='1922',
     month='декабря',
     day='10'
 ),
 Date(
     year='1945',
     month='июля',
     day='16'
 ),
 Date(
     year='1945',
     month='августа',
     day='6'
 ),
 Date(
     year='1955',
     month='апреля',
     day='18'
 ),
 Date(
     year='1955',
     month='апреля',
     day='19'
 ),
 Date(
     year='1903',
     month='января',
     day='6'
 ),
 Date(
     year='2009',
     month='июня',
     day='21'
 ),
 Date(
     year=None,
     month='ноября',
   

In [11]:
from yargy.predicates import in_
Action = fact(
    'Action',
    ['titles', 'year']
)

Y = morph_pipeline(['год'])

TITLE1 = rule(
    Y,
    dictionary({
        'Альберт Эйнштейн',
        'Эйнштейн',
        'он'
    }),
    not_(eq('.')).repeatable(),
)


YEAR = and_(
    gte(1800),
    lte(2100)
)
DATE = rule(
    YEAR.interpretation(
        Action.year
    )
)

ACTION = rule(
    DATE.interpretation(
        Action.year
    ),
    TITLE1.interpretation(
        Action.titles
    )
).interpretation(
    Action
)

parser = Parser(ACTION)
matches = []
for sent in sents:
    for match in parser.findall(sent):
        matches.append(match.fact)
for m in matches:
    print(f"{m.year} год - {m.titles[5:]}")

1842 год - он сменил фамилию на Кох) и Йетты Бернхаймер[14]
1894 год - Эйнштейны переехали из Мюнхена в итальянский город Павию, близ Милана, куда братья Герман и Якоб перевели свою фирму
1895 год - он присоединился к своей семье в Павии
1896 год - он успешно сдал все выпускные экзамены в школе, за исключением экзамена по французскому языку, и получил аттестат, а в октябре 1896 года был принят в Политехникум на педагогический факультет[18]
1900 год - Эйнштейн окончил Политехникум, получив диплом преподавателя математики и физики
1902 год - он не мог найти постоянное место работы — даже школьным учителем[19]
1901 год - Эйнштейн закончил работу над докторской диссертацией, посвящённой молекулярным силам в газах, и передал её в Цюрихский университет
1902 год - Эйнштейн забрал её обратно[22]
1903 год - он стал постоянным работником Бюро
1902 год - Эйнштейн получил известие из Италии о болезни отца; Герман Эйнштейн умер спустя несколько дней после приезда сына
1904 год - Эйнштейн сотруднича